# MiSTer FPGA D93 simulator

MiSTer FPGA offers a gamma scaler that can optionally scale separate R, G and B output channels for an greyscale input value. 

This is not a full 3D lut, and can only apply to an input greyscale value, i.e.: where the R, G and B values are always the same across the 0-255 range. 

As such, R,G,B primary chromaticities have no bearing, and any matrix selected will produce the same output due to only the white point being shifted. 

Output files with the `.txt` extension will be placed in the `mister` folder of this repository. 

Copy these to your `/media/fat/gamma` foder on your MiSTer, and select them from the Video Processing -> Gamma Correction menu per core. 

This only works with the HDMI/VGA scaler options, not with direct video or the analogue IO boards. 

The notebook `compare_gamut.ipynb` in the `jupyter` folder of this repo visually demonstrates D93 versus D65 inside a standard SDR container, and where clipping occurs. 

When scaled down to around 74.54%, the white point of D93 will fit inside D65.  This sacrifices some brightness in order to avoid clipping. 

This notebook generates MiSTer FPGA gamma options for 75-100% scaled D93, where 100% has no brightness loss, but will clip the Blue channel as much as 12%.  Smaller percentages clip Blue less, down to 75% which doesn't clip. (Green and Red never clip in any). 

Gamma scaling is also on offer, with assumed source gammas of 1.8 to 2.6.  Destination gamma 2.2 is assumed "normal" and won't change the gamma compared to the source. 

In [ ]:
import numpy as np
import PIL
import csv

In [ ]:
## Read in the matrices from our CSV files
import csv
with open('../csv/matrix_XYZ.csv') as f:
    matrix_dict = csv.DictReader(f)
    for cspace in matrix_dict:
        # Our destination matrix for computers will be sRGB
        if cspace['col_id'] == 'BT709':
            XYZ_to_BT709 = np.array([float(cspace['Mdst0']), float(cspace['Mdst1']), float(cspace['Mdst2']),
                                    float(cspace['Mdst3']), float(cspace['Mdst4']), float(cspace['Mdst5']),
                                    float(cspace['Mdst6']), float(cspace['Mdst7']), float(cspace['Mdst8'])])
            XYZ_to_BT709 = np.reshape(XYZ_to_BT709, (3,3))
        # Source matrices only from here
        elif cspace['col_id'] == 'ARIBTRB9v1':
            ARIBTRB9v1_to_XYZ = np.array([float(cspace['Msrc0']), float(cspace['Msrc1']), float(cspace['Msrc2']),
                                          float(cspace['Msrc3']), float(cspace['Msrc4']), float(cspace['Msrc5']),
                                          float(cspace['Msrc6']), float(cspace['Msrc7']), float(cspace['Msrc8'])])
            ARIBTRB9v1_to_XYZ = np.reshape(ARIBTRB9v1_to_XYZ, (3,3))
        elif cspace['col_id'] == 'sRGB_D93':
            sRGB_D93_to_XYZ = np.array([float(cspace['Msrc0']), float(cspace['Msrc1']), float(cspace['Msrc2']),
                                        float(cspace['Msrc3']), float(cspace['Msrc4']), float(cspace['Msrc5']),
                                        float(cspace['Msrc6']), float(cspace['Msrc7']), float(cspace['Msrc8'])])
            sRGB_D93_to_XYZ = np.reshape(sRGB_D93_to_XYZ, (3,3))
        elif cspace['col_id'] == 'Raney_PVM_20M2U':
            Raney_PVM_20M2U_to_XYZ = np.array([float(cspace['Msrc0']), float(cspace['Msrc1']), float(cspace['Msrc2']),
                                               float(cspace['Msrc3']), float(cspace['Msrc4']), float(cspace['Msrc5']),
                                               float(cspace['Msrc6']), float(cspace['Msrc7']), float(cspace['Msrc8'])])
            Raney_PVM_20M2U_to_XYZ = np.reshape(Raney_PVM_20M2U_to_XYZ, (3,3))
        elif cspace['col_id'] == 'Raney_PVM_20L2MDU':
            Raney_PVM_20L2MDU_XYZ = np.array([float(cspace['Msrc0']), float(cspace['Msrc1']), float(cspace['Msrc2']),
                                              float(cspace['Msrc3']), float(cspace['Msrc4']), float(cspace['Msrc5']),
                                              float(cspace['Msrc6']), float(cspace['Msrc7']), float(cspace['Msrc8'])])
            Raney_PVM_20L2MDU_XYZ = np.reshape(Raney_PVM_20L2MDU_XYZ, (3,3))





In [ ]:
for percent in range(75, 101, 5):
    # Scaling to 74.54% is where D93 stops clipping inside a D65 container
    # So we'll create 75% to 100% options in 5% jumps
    filename = "../mister/D93_"+str(percent).zfill(3)+"_percent.txt"
    scale = float(percent)/100.00
    for gammasuffix in range(0, 9, 2):
        # Offer gamma scaling as well, from gamma 1.8 to 2.6
        # Old Apple stuff was gamma 1.8
        # SDTV, PCs, sRGB and Rec.601 are approximately gamma 2.2
        # HDTV, Rec.701 uses BT.1886 which is gamma 2.4
        # These may also help in general to compensate for brightness loss in the scaled options
        srcgamma = 1.8 + (float(gammasuffix)/10)
        # Create the text file
        filename = "../mister/D93_"+str(percent).zfill(3)+"_percent_gamma_"+str(srcgamma)+".txt"
        with open(filename, 'w') as f:
            f.write("# Simulate D93 white point inside a D65 container. \n")
            f.write("# Scaled down to "+str(percent)+"%, where 75% has no Blue channel clipping. \n")
            f.write("# Higher percentages are brighter, but clip more Blue channel. \n")
            f.write("# Assumed source gamma "+str(srcgamma)+", and destination gamma 2.2 \n")
            f.write("# Gamma 2.2 is unchanged from the source. Smaller is brighter in the mid range, larger is darker in the mid range. \n")
            f.write("# by Dan Mons \n")
            f.write("# https://github.com/danmons/colour_matrix_adaptations \n")
            for i in range(0, 256):
                # create the RGB triplet
                # MiSTer gamma correction only works on the greyscale
                # So this will always be [i,i,i]
                in_rgb = np.array([i,i,i])
                # Convert to a normalised float between 0.0 and 1.0
                in_rgb = in_rgb.astype(float)/255.0
                # Remove gamma correction using our assumed source gamma
                in_rgb = in_rgb**srcgamma
                # Apply the D93 transformation into XYZ
                out_rgb = np.matmul(sRGB_D93_to_XYZ, in_rgb)
                # Scale by our desired percentage
                out_rgb = out_rgb*scale
                # Convert from XYZ back to our destination output
                out_rgb = np.matmul(XYZ_to_BT709, out_rgb)
                # Apply the destination gamma
                out_rgb = out_rgb**(1/2.2)
                # Convert back to 0-255 int, and clip if necessary
                out_rgb = out_rgb*255.0
                out_rgb = np.round(out_rgb)
                out_rgb = np.clip(out_rgb, 0, 255)
                out_rgb = out_rgb.astype(int)
                #print(out_rgb)
                out_r = str(out_rgb[0])
                out_g = str(out_rgb[1])
                out_b = str(out_rgb[2])
                out = out_r + ',' + out_g + ',' + out_b + '\n'
                # Write the new RGB triplet 
                f.write(out)
            f.close()


